### Execture cells with SHIFT-ENTER

This cell loads the necessary libraries. The logging output of datajoint will disappear in future versions. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import datajoint as dj
from pprint import pprint 
dj.config['loglevel'] = 'INFO'
from pipeline import rf, pre, utils, trk
import numpy as np

### Look at possible keys (can be skipped)

This cell can be used to find the key that is to be populated. 

In [ ]:
rf.Eye().unpopulated()

### Set the values of key to the desired session 

Change the values of the dictionary to target the desired session and scan. `rf.Eye().new_eye(key)` will load the time stamps and a few frames to select the ROI.

In [ ]:
key = dict(animal_id=2054, session=1, scan_idx=2)
eye_time, frames = rf.Eye().new_eye(key)

### Execute next cell and draw a rectangle around ROI

This cell selects the region of interest. Draw a rectangle around the entire eye. When done simply execute the next cell below. 

In [ ]:
rg = utils.ROIGrabber(frames.mean(axis=2))

### Insert results into database

This cell inserts the timestamps and the ROI into the database. Normally, a tracking minion should then start to track the video.

In [ ]:
key['eye_time'] = eye_time
key['eye_roi'] = rg.roi
pprint(key)
rf.Eye().insert1(key)


# Insert ROI for automatic tracking

In [ ]:
rg = utils.ROIGrabber(frames.mean(axis=2))

In [ ]:
trk.Roi()

If you are happy with the ROI, execute the next cell. 

In [ ]:
roi = rg.roi
x_roi_min = min(roi[0], roi[2])
x_roi_max = max(roi[0], roi[2])
y_roi_min = min(roi[1], roi[3])
y_roi_max = max(roi[1], roi[3])
key['x_roi_min'] = x_roi_min
key['y_roi_min'] = y_roi_min
key['x_roi_max'] = x_roi_max
key['y_roi_max'] = y_roi_max

key.pop('eye_roi')
print('Inserting the following tuple', key)
trk.Roi().insert1(key)
